In [1]:
import sys
import os
rise_sfl_path = "/content/RISE-SFL/RISE-SFL"  # Replace with the actual path to your RISE-SFL folder
sys.path.append(rise_sfl_path)
from utils.utils import *
# From masker/generation.py
from masker.generation import SFL

import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torchvision import transforms, datasets
import torchvision.models as models
from torch.utils.data import DataLoader
from PIL import Image
import json
from torch.nn.functional import softmax, conv2d
import os
import requests
import zipfile

In [2]:
cudnn.benchmark = True
# Load black box model for explanations
model = models.resnet50(True)
model = nn.Sequential(model, nn.Softmax(dim=1))
model = model.eval()
model = model.cuda()

for p in model.parameters():
    p.requires_grad = False


c:\Users\eagua\anaconda3\envs\torch_cuda\Lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
c:\Users\eagua\anaconda3\envs\torch_cuda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
from masker.batch import SFL_batch

input_size = (224,224)
sfl_batch = SFL_batch(model, input_size)
image_folder = r'C:\Users\eagua\OneDrive\Documents\GitHub\6CCS3COV-7CCSMCVI-Computer-Vision\RISE-SFL\test_data'
N = 100
s = 8
p1 = 0.5
masks, sampled_tensors, target_list = sfl_batch.generate_batch_images(image_folder, N, s, p1)

Processing images:   0%|          | 0/19 [00:00<?, ?it/s]c:\Users\eagua\OneDrive\Documents\GitHub\6CCS3COV-7CCSMCVI-Computer-Vision\RISE-SFL\RISE-SFL\masker\generation.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cell_size = torch.ceil(torch.tensor(self.input_size).float() / s)
Processing images: 100%|██████████| 19/19 [02:52<00:00,  9.07s/it]


In [4]:
def generate_pass_fail_list(N):
    return ["Pass" if i % 2 == 0 else "Fail" for i in range(N)]

pass_fail_list = generate_pass_fail_list(N)

from visuals.plots import SFLVisualizer

In [5]:
#If we want to observe any of the images that are being explained, we just need to 
visualizer = SFLVisualizer()
visualizer.interactive_mutant_visualization(sampled_tensors[0], pass_fail_list)


interactive(children=(IntSlider(value=0, description='Mutant Index:', max=99), Output()), _dom_classes=('widge…

In [6]:
from sfl_techniques.sfl import RelevanceScore
#A dictionary will be return for the image to be explained

# For each image, after returning the dictionary, we need to use causal metrics to calculate which one is more efficient
relevance_score_calculator = RelevanceScore(device='cuda')

pixel_datasets = []
ochiai_array = []
zoltar_array = []
tarantula_array = []
wong1_array = []
for i in range(19):
    pixel_dataset, ochiai, tarantula, zoltar, wong1 = relevance_score_calculator.run(sampled_tensors[i], masks[i], N)
    pixel_datasets.append(pixel_dataset)
    ochiai_array.append(ochiai)
    zoltar_array.append(zoltar)
    tarantula_array.append(tarantula)
    wong1_array.append(wong1)

In [7]:
from visuals.saliency_maps.saliency import SaliencyMapVisualizer

# List all files in the directory
all_files = os.listdir(image_folder)

# Filter for .jpg and .png files
image_files = [file for file in all_files if file.lower().endswith(('.jpg', '.png'))]
'''
image_path = os.path.join(image_folder, image_files[0])

saliency_map = SaliencyMapVisualizer(image_path)
saliency_map.visualize_pixel_scores(pixel_datasets[0],ins='')
'''

"\nimage_path = os.path.join(image_folder, image_files[0])\n\nsaliency_map = SaliencyMapVisualizer(image_path)\nsaliency_map.visualize_pixel_scores(pixel_datasets[0],ins='')\n"

In [8]:
rise_sfl_path = r"C:\Users\eagua\OneDrive\Documents\GitHub\6CCS3COV-7CCSMCVI-Computer-Vision\metrics-saliency-maps"  # Replace with the actual path to your RISE-SFL folder
sys.path.append(rise_sfl_path)
#saliency_maps = extract_specific_saliency_maps(pixel_datasets[0])

from saliency_maps_metrics.multi_step_metrics import Deletion, Insertion, compute_correlation, compute_auc_metric
from saliency_maps_metrics.data_replace import select_data_replace_method

deletion_metric = Deletion(data_replace_method="black", bound_max_step=True, batch_size=1, max_step_nb=14*14, cumulative=True)
insertion_metric = Insertion(data_replace_method="blur", bound_max_step=True, batch_size=1, max_step_nb=14*14, cumulative=True)



In [9]:

from tabulate import tabulate  
device='cuda'
'''
n = 2
image_path = os.path.join(image_folder, image_files[n])
img = read_tensor(image_path).to(device)
# Create instances of Deletion and Insertion metrics
model.to(device)
target_class = target_list[n]
# Initialize lists to store results

# Function to calculate sparsity
def calculate_sparsity(saliency_map):
    max_value = np.max(saliency_map)
    mean_value = np.mean(saliency_map)
    return max_value / mean_value if mean_value != 0 else float('inf')

# Create a dictionary of saliency maps
saliency_maps = {
    "Wong1": wong1_array[n][0],
    "Tarantula": tarantula_array[n][0],
    "Zoltar": zoltar_array[n][0],
    "Ochiai": ochiai_array[n][0]
}

device='cuda'
# Create instances of Deletion and Insertion metrics
deletion_metric = Deletion(data_replace_method="black", bound_max_step=True, batch_size=1, max_step_nb=14*14, cumulative=True)
insertion_metric = Insertion(data_replace_method="blur", bound_max_step=True, batch_size=1, max_step_nb=14*14, cumulative=True)

# Initialize lists to store results
results = []

# Loop through each saliency map
for name, saliency_map in saliency_maps.items():
    # Convert saliency map to PyTorch tensor and add batch and channel dimensions
    saliency_map_tensor = torch.from_numpy(saliency_map).float().unsqueeze(0).unsqueeze(0)
    saliency_map_tensor = saliency_map_tensor.to(device)

    # Compute Deletion metrics
    deletion_result = deletion_metric(model, img, saliency_map_tensor, class_to_explain_list=[target_class])
    dauc_mean = deletion_result["dauc"]
    dc_mean = deletion_result["dc"]

    # Compute Insertion metrics
    insertion_result = insertion_metric(model, img, saliency_map_tensor, class_to_explain_list=[target_class])
    iauc_mean = insertion_result["iauc"]
    ic_mean = insertion_result["ic"]
    
    # Calculate sparsity
    sparsity = calculate_sparsity(saliency_map)
    # Store results
    results.append([name, dauc_mean, dc_mean, iauc_mean, ic_mean, sparsity])

# Create a table using tabulate
headers = ["Saliency Map", "DAUC", "DC", "IAUC", "IC","Sparsity"]
table = tabulate(results, headers=headers, tablefmt="grid", floatfmt=".4f")

# Print the table
print(f"Metrics for target class: {target_class}")
print(table)
'''

'\nn = 2\nimage_path = os.path.join(image_folder, image_files[n])\nimg = read_tensor(image_path).to(device)\n# Create instances of Deletion and Insertion metrics\nmodel.to(device)\ntarget_class = target_list[n]\n# Initialize lists to store results\n\n# Function to calculate sparsity\ndef calculate_sparsity(saliency_map):\n    max_value = np.max(saliency_map)\n    mean_value = np.mean(saliency_map)\n    return max_value / mean_value if mean_value != 0 else float(\'inf\')\n\n# Create a dictionary of saliency maps\nsaliency_maps = {\n    "Wong1": wong1_array[n][0],\n    "Tarantula": tarantula_array[n][0],\n    "Zoltar": zoltar_array[n][0],\n    "Ochiai": ochiai_array[n][0]\n}\n\ndevice=\'cuda\'\n# Create instances of Deletion and Insertion metrics\ndeletion_metric = Deletion(data_replace_method="black", bound_max_step=True, batch_size=1, max_step_nb=14*14, cumulative=True)\ninsertion_metric = Insertion(data_replace_method="blur", bound_max_step=True, batch_size=1, max_step_nb=14*14, cumul

In [10]:
from tabulate import tabulate
import torch
import numpy as np
import os
from tqdm import tqdm

device = 'cuda'
model.to(device)


def get_all_image_files(directory):
    image_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_files.append(os.path.join(root, file))
    return image_files

# Use the function to get all image files
image_files = get_all_image_files(image_folder)

# Check if there are any image files in the directory
if not image_files:
    print("No image files found in the specified directory or its subdirectories.")
else:
    print(f"Found {len(image_files)} image files.")

    device = 'cuda'
    model.to(device)

# Function to calculate sparsity
def calculate_sparsity(saliency_map):
    max_value = np.max(saliency_map)
    mean_value = np.mean(saliency_map)
    return max_value / mean_value if mean_value != 0 else float('inf')

# Create instances of Deletion and Insertion metrics
deletion_metric = Deletion(data_replace_method="black", bound_max_step=True, batch_size=1, max_step_nb=14*14, cumulative=True)
insertion_metric = Insertion(data_replace_method="blur", bound_max_step=True, batch_size=1, max_step_nb=14*14, cumulative=True)

# Initialize dictionary to store cumulative results
cumulative_results = {
    "Wong1": {"DAUC": 0, "DC": 0, "IAUC": 0, "IC": 0, "Sparsity": 0},
    "Tarantula": {"DAUC": 0, "DC": 0, "IAUC": 0, "IC": 0, "Sparsity": 0},
    "Zoltar": {"DAUC": 0, "DC": 0, "IAUC": 0, "IC": 0, "Sparsity": 0},
    "Ochiai": {"DAUC": 0, "DC": 0, "IAUC": 0, "IC": 0, "Sparsity": 0}
}

# Get list of image files
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Process each image
for i, (image_file, target_class) in enumerate(zip(image_files, target_list)):
    image_path = os.path.join(image_folder, image_file)
    img = read_tensor(image_path).to(device)

    # Create a dictionary of saliency maps for this image
    saliency_maps = {
        "Wong1": wong1_array[i][0],
        "Tarantula": tarantula_array[i][0],
        "Zoltar": zoltar_array[i][0],
        "Ochiai": ochiai_array[i][0]
    }

    # Process each saliency map for this image
    for name, saliency_map in saliency_maps.items():
        saliency_map_tensor = torch.from_numpy(saliency_map).float().unsqueeze(0).unsqueeze(0).to(device)

        # Compute Deletion metrics
        deletion_result = deletion_metric(model, img, saliency_map_tensor, class_to_explain_list=[target_class])
        cumulative_results[name]["DAUC"] += deletion_result["dauc"]
        cumulative_results[name]["DC"] += deletion_result["dc"]

        # Compute Insertion metrics
        insertion_result = insertion_metric(model, img, saliency_map_tensor, class_to_explain_list=[target_class])
        cumulative_results[name]["IAUC"] += insertion_result["iauc"]
        cumulative_results[name]["IC"] += insertion_result["ic"]

        # Calculate sparsity
        cumulative_results[name]["Sparsity"] += calculate_sparsity(saliency_map)

# Calculate averages
num_images = len(image_files)
results = []
for name, metrics in cumulative_results.items():
    avg_metrics = [name] + [metrics[key] / num_images for key in ["DAUC", "DC", "IAUC", "IC", "Sparsity"]]
    results.append(avg_metrics)

# Create a table using tabulate
headers = ["Saliency Map", "Avg DAUC", "Avg DC", "Avg IAUC", "Avg IC", "Avg Sparsity"]
table = tabulate(results, headers=headers, tablefmt="grid", floatfmt=".4f")

# Print the table
print(f"Average Metrics for {num_images} images:")
print(table)

Found 19 image files.
Average Metrics for 19 images:
+----------------+------------+----------+------------+----------+----------------+
| Saliency Map   |   Avg DAUC |   Avg DC |   Avg IAUC |   Avg IC |   Avg Sparsity |
+================+============+==========+============+==========+================+
| Wong1          |     0.1322 |   0.2064 |     0.7327 |   0.1107 |         2.5028 |
+----------------+------------+----------+------------+----------+----------------+
| Tarantula      |     0.1332 |   0.2148 |     0.7391 |   0.1256 |         2.8643 |
+----------------+------------+----------+------------+----------+----------------+
| Zoltar         |     0.1293 |   0.1633 |     0.7245 |   0.1131 |         1.5394 |
+----------------+------------+----------+------------+----------+----------------+
| Ochiai         |     0.1294 |   0.2091 |     0.7256 |   0.1361 |         2.7933 |
+----------------+------------+----------+------------+----------+----------------+
